In [77]:
%load_ext autoreload
%autoreload 2
from models import aggregate_results_from_subfolders

results_df = aggregate_results_from_subfolders('.')
results_df.to_csv('results.csv', index=False)
print(f"{len(results_df)} records")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
34 records
